# Gates Foundation



In [14]:
import polars as pl
from oic_scrape.items import AwardItem
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from io import StringIO

In [3]:
session = requests.Session()

In [7]:
from math import e


grants_web_url = "https://www.gatesfoundation.org/about/committed-grants"
grants_page = session.get(grants_web_url)
grants_page.raise_for_status()

soup = BeautifulSoup(grants_page.text, 'html.parser')
anchor = soup.select_one('.arrow-link--download')
try:
    href = anchor['href']
except ValueError as e:
    raise ValueError(f"No grants download URL found: {e}")

latest_grant_csv_url = urljoin(grants_web_url, href)

In [8]:
df = pl.read_csv(latest_grant_csv_url)

HTTPError: HTTP Error 403: Forbidden

In [10]:
csv = session.get(latest_grant_csv_url)
csv.raise_for_status()

In [15]:
csv_contents = StringIO(csv.text)

In [17]:
df = pd.read_csv(csv_contents)

EmptyDataError: No columns to parse from file

In [18]:
csv_contents